In [1]:
#| include: false

import plotly.express as px
import plotly.graph_objects as go

import duckdb
import pandas as pd
%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

%sql CREATE OR REPLACE TABLE user_actions AS SELECT * FROM read_csv('00_data/sql/user_actions.csv', header=True, columns={'user_id': 'INT', 'order_id': 'INT', 'action': 'VARCHAR', 'time': 'TIMESTAMP'}, timestampformat='%d/%m/%y %H:%M');
%sql CREATE OR REPLACE TABLE courier_actions AS SELECT * FROM read_csv('00_data/sql/courier_actions.csv', header=True, columns={'courier_id': 'INT', 'order_id': 'INT', 'action': 'VARCHAR', 'time': 'TIMESTAMP'}, timestampformat='%d/%m/%y %H:%M');
%sql CREATE OR REPLACE TABLE orders AS SELECT * FROM read_csv('00_data/sql/orders.csv', header=True, columns={'order_id': 'INT', 'creation_time': 'TIMESTAMP', 'product_ids': 'INT[]'}, timestampformat='%d/%m/%y %H:%M');
%sql CREATE OR REPLACE TABLE users AS SELECT * FROM read_csv('00_data/sql/users.csv', header=True, columns={'user_id': 'INT', 'birth_date': 'DATE', 'sex': 'VARCHAR'}, dateformat='%d/%m/%y');
%sql CREATE OR REPLACE TABLE couriers AS SELECT * FROM read_csv('00_data/sql/couriers.csv', header=True, columns={'courier_id': 'INT', 'birth_date': 'DATE', 'sex': 'VARCHAR'}, dateformat='%d/%m/%y');
%sql CREATE OR REPLACE TABLE products AS SELECT * FROM read_csv('00_data/sql/products.csv', header=True, columns={'product_id': 'INT', 'name': 'VARCHAR', 'price': 'DOUBLE'});

,Count
0,87


# Практичні задачі

У практичній частині ми продовжимо аналізувати наш сервіс та розрахуємо кілька важливих показників, що характеризують його роботу. Вам буде запропоновано написати кілька SQL-запитів та візуалізувати їх.

## Задача 1

Для початку давайте проаналізуємо, наскільки швидко зростає аудиторія нашого сервісу та подивимося на динаміку числа користувачів та кур'єрів.

::::: {.callout-note icon=false}
## Завдання
:::: {#exr-sql-analytic-01}
<br>
Для кожного дня, представленого в таблицях `user_actions` та `courier_actions`, розрахуйте такі показники:

1. Число нових користувачів.
2. Число нових кур'єрів.
3. Загальна кількість користувачів на сьогодні.
4. Загальна кількість кур'єрів на цей день.

Колонки з показниками назвіть відповідно `new_users`, `new_couriers`, `total_users`, `total_couriers`. Колонку з датами назвіть `date`. Простежте, щоб показники були виражені **цілими числами**. Результат має бути відсортований за зростанням дати.

Поля в результуючій таблиці: `date`, `new_users`, `new_couriers`, `total_users`, `total_couriers`

**Пояснення:**

Новими вважатимемо тих користувачів та кур'єрів, які в цей день здійснили свою першу дію в нашому сервісі. Загальна кількість користувачів/кур'єрів на поточний день — це результат додавання числа нових користувачів/кур'єрів у поточний день зі значеннями аналогічного показника всіх попередніх днів.

::: {.callout-note collapse="true"}
## Підказка на випадок, якщо зовсім не виходить

Щоб порахувати кількість нових користувачів/кур'єрів на кожну дату спочатку необхідно для кожного користувача/кур'єра визначити мінімальну дату, тобто дату першої дії в нашому сервісі. Потім потрібно провести групування і для кожної отриманої дати порахувати кількість користувачів/кур'єрів із цією датою. Далі до отриманої таблиці достатньо застосувати віконну функцію і кожної дати обчислити накопичувальну суму числа користувачів/кур'єрів.
:::


::::
:::::

In [2]:
#| code-fold: true
#| code-summary: "Рішення"

%%sql
SELECT start_date as date,
       new_users,
       new_couriers,
       (sum(new_users) OVER (ORDER BY start_date))::int as total_users,
       (sum(new_couriers) OVER (ORDER BY start_date))::int as total_couriers
FROM   (SELECT start_date,
               count(courier_id) as new_couriers
        FROM   (SELECT courier_id,
                       min(time::date) as start_date
                FROM   courier_actions
                GROUP BY courier_id) t1
        GROUP BY start_date) t2
    LEFT JOIN (SELECT start_date,
                      count(user_id) as new_users
               FROM   (SELECT user_id,
                              min(time::date) as start_date
                       FROM   user_actions
                       GROUP BY user_id) t3
               GROUP BY start_date) t4 using (start_date)

,date,new_users,new_couriers,total_users,total_couriers
0,2022-08-24,134,95,134,95
1,2022-08-25,829,242,963,337
2,2022-08-26,1017,219,1980,556
3,2022-08-27,1225,186,3205,742
4,2022-08-28,1487,213,4692,955
5,2022-08-29,1309,109,6001,1064
6,2022-08-30,1204,127,7205,1191
7,2022-08-31,1420,195,8625,1386
8,2022-09-01,1513,205,10138,1591
9,2022-09-02,1887,229,12025,1820


Запишемо результат запиту у змінну `results`.

In [3]:
#| code-fold: true

%%sql
results << SELECT start_date as date,
       new_users,
       new_couriers,
       (sum(new_users) OVER (ORDER BY start_date))::int as total_users,
       (sum(new_couriers) OVER (ORDER BY start_date))::int as total_couriers
FROM   (SELECT start_date,
               count(courier_id) as new_couriers
        FROM   (SELECT courier_id,
                       min(time::date) as start_date
                FROM   courier_actions
                GROUP BY courier_id) t1
        GROUP BY start_date) t2
    LEFT JOIN (SELECT start_date,
                      count(user_id) as new_users
               FROM   (SELECT user_id,
                              min(time::date) as start_date
                       FROM   user_actions
                       GROUP BY user_id) t3
               GROUP BY start_date) t4 using (start_date)

Побудуємо візуалізацію за отриманими даними:

In [4]:
# | label: fig-sql-analytic-01
# | fig-cap: "Графік за результатами SQL-запиту"
# | fig-subcap:
# |   - "Динаміка загальної кількості користувачів"
# |   - "Динаміка нових користувачів та кур'єрів"
# | column: screen-inset-shaded
# | layout-nrow: 1

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=results.date, y=results.total_users, mode="lines+markers", name="total_users"
    )
)
fig.add_trace(
    go.Scatter(
        x=results.date,
        y=results.total_couriers,
        mode="lines+markers",
        name="total_couriers",
    )
)
fig.show()

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=results.date, y=results.new_users, mode="lines+markers", name="new_users"
    )
)
fig.add_trace(
    go.Scatter(
        x=results.date,
        y=results.new_couriers,
        mode="lines+markers",
        name="new_couriers",
    )
)
fig.show()

Проаналізуйте побудовані графіки та спробуйте відповісти на такі питання:

1. Що зростає швидше: кількість користувачів чи кількість кур'єрів?
2. Наскільки стабільні показники числа нових користувачів та кур'єрів? Чи немає даних таких днів, коли показники сильно вибивалися із загальної динаміки?
3. Чи можна сказати, що кількість нових кур'єрів більш стабільна, ніж кількість нових користувачів?

## Задача 2

Аналізуючи динаміку показників із попереднього завдання, ви могли помітити, що порівнювати абсолютні значення не дуже зручно. Давайте порахуємо динаміку показників у відносних величинах.

::::: {.callout-note icon=false}
## Завдання
:::: {#exr-sql-analytic-02}
<br>
Доповніть запит із попереднього завдання і тепер для кожного дня, поданого в таблицях `user_actions` та `courier_actions`, додатково розрахуйте такі показники:

1. Приріст числа нових користувачів.
2. Приріст числа нових кур'єрів.
3. Приріст загальної кількості користувачів.
4. Приріст загальної кількості кур'єрів.

Показники, розраховані на попередньому кроці, також включіть у результуючу таблицю.

Колонки з новими показниками назвіть відповідно `new_users_change`, `new_couriers_change`, `total_users_growth`, `total_couriers_growth`. Колонку з датами назвіть `date`.

Усі показники приросту рахуйте **у відсотках** відносно попереднього дня. Під час розрахунку показників округляйте значення **до двох знаків після коми**.

Результуюча таблиця має бути відсортована за зростанням дати.

Поля в результуючій таблиці: `date`, `new_users`, `new_couriers`, `total_users`, `total_couriers`, `new_users_change`, `new_couriers_change`, `total_users_growth`, `total_couriers_growth`

**Пояснення:**

Не забувайте при діленні заздалегідь наводити значення до потрібного типу даних. Пропущені значення приростів для першої дати не заповнюйте - просто залиште поля в цьому рядку порожніми.

::: {.callout-note collapse="true"}
## Підказка на випадок, якщо зовсім не виходить

Для розрахунку приростів скористайтесь віконними функціями та функціями зсунення.
:::


::::
:::::

In [5]:
#| code-fold: true
#| code-summary: "Рішення"
#| column: page-inset

%%sql
SELECT date,
       new_users,
       new_couriers,
       total_users,
       total_couriers,
       round(100 * (new_users - lag(new_users, 1) OVER (ORDER BY date)) / lag(new_users, 1) OVER (ORDER BY date)::decimal,
             2) as new_users_change,
       round(100 * (new_couriers - lag(new_couriers, 1) OVER (ORDER BY date)) / lag(new_couriers, 1) OVER (ORDER BY date)::decimal,
             2) as new_couriers_change,
       round(100 * new_users::decimal / lag(total_users, 1) OVER (ORDER BY date),
             2) as total_users_growth,
       round(100 * new_couriers::decimal / lag(total_couriers, 1) OVER (ORDER BY date),
             2) as total_couriers_growth
FROM   (SELECT start_date as date,
               new_users,
               new_couriers,
               (sum(new_users) OVER (ORDER BY start_date))::int as total_users,
               (sum(new_couriers) OVER (ORDER BY start_date))::int as total_couriers
        FROM   (SELECT start_date,
                       count(courier_id) as new_couriers
                FROM   (SELECT courier_id,
                               min(time::date) as start_date
                        FROM   courier_actions
                        GROUP BY courier_id) t1
                GROUP BY start_date) t2
            LEFT JOIN (SELECT start_date,
                              count(user_id) as new_users
                       FROM   (SELECT user_id,
                                      min(time::date) as start_date
                               FROM   user_actions
                               GROUP BY user_id) t3
                       GROUP BY start_date) t4 using (start_date)) t5

,date,new_users,new_couriers,total_users,total_couriers,new_users_change,new_couriers_change,total_users_growth,total_couriers_growth
0,2022-08-24,134,95,134,95,NaN,NaN,NaN,NaN
1,2022-08-25,829,242,963,337,518.66,154.74,618.66,254.74
2,2022-08-26,1017,219,1980,556,22.68,-9.50,105.61,64.99
3,2022-08-27,1225,186,3205,742,20.45,-15.07,61.87,33.45
4,2022-08-28,1487,213,4692,955,21.39,14.52,46.40,28.71
5,2022-08-29,1309,109,6001,1064,-11.97,-48.83,27.90,11.41
6,2022-08-30,1204,127,7205,1191,-8.02,16.51,20.06,11.94
7,2022-08-31,1420,195,8625,1386,17.94,53.54,19.71,16.37
8,2022-09-01,1513,205,10138,1591,6.55,5.13,17.54,14.79
9,2022-09-02,1887,229,12025,1820,24.72,11.71,18.61,14.39


Запишемо результат запиту у змінну `results`.

In [6]:
#| code-fold: true

%%sql
results << SELECT date,
       new_users,
       new_couriers,
       total_users,
       total_couriers,
       round(100 * (new_users - lag(new_users, 1) OVER (ORDER BY date)) / lag(new_users, 1) OVER (ORDER BY date)::decimal,
             2) as new_users_change,
       round(100 * (new_couriers - lag(new_couriers, 1) OVER (ORDER BY date)) / lag(new_couriers, 1) OVER (ORDER BY date)::decimal,
             2) as new_couriers_change,
       round(100 * new_users::decimal / lag(total_users, 1) OVER (ORDER BY date),
             2) as total_users_growth,
       round(100 * new_couriers::decimal / lag(total_couriers, 1) OVER (ORDER BY date),
             2) as total_couriers_growth
FROM   (SELECT start_date as date,
               new_users,
               new_couriers,
               (sum(new_users) OVER (ORDER BY start_date))::int as total_users,
               (sum(new_couriers) OVER (ORDER BY start_date))::int as total_couriers
        FROM   (SELECT start_date,
                       count(courier_id) as new_couriers
                FROM   (SELECT courier_id,
                               min(time::date) as start_date
                        FROM   courier_actions
                        GROUP BY courier_id) t1
                GROUP BY start_date) t2
            LEFT JOIN (SELECT start_date,
                              count(user_id) as new_users
                       FROM   (SELECT user_id,
                                      min(time::date) as start_date
                               FROM   user_actions
                               GROUP BY user_id) t3
                       GROUP BY start_date) t4 using (start_date)) t5

Побудуємо візуалізацію за отриманими даними:

In [7]:
# | label: fig-sql-analytic-02
# | fig-cap: "Графік за результатами SQL-запиту"
# | fig-subcap:
# |   - "Динаміка приросту числа нових користувачів та кур'єрів"
# |   - "Динаміка приросту загальної кількості користувачів та кур'єрів"
# | column: screen-inset-shaded
# | layout-nrow: 1

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=results.date,
        y=results.new_users_change,
        name="new_users_change",
        marker_color="indianred",
    )
)
fig.add_trace(
    go.Bar(
        x=results.date,
        y=results.new_couriers_change,
        name="new_couriers_change",
        marker_color="lightsalmon",
    )
)

fig.update_layout(barmode="group", xaxis_tickangle=-45)
fig.show()

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=results.date,
        y=results.total_users_growth,
        name="total_users_growth",
        marker_color="indianred",
    )
)
fig.add_trace(
    go.Bar(
        x=results.date,
        y=results.total_couriers_growth,
        name="total_couriers_growth",
        marker_color="lightsalmon",
    )
)

fig.update_layout(barmode="group", xaxis_tickangle=-45)
fig.show()

Проаналізуйте побудовані графіки та спробуйте відповісти на такі питання:

1. Як змінилися темпи приросту загальної кількості користувачів і кур'єрів за проміжок часу, що розглядається? Яка загалом динаміка цих показників: вони ростуть чи, навпаки, згасають?
2. У які дні темп приросту числа нових кур'єрів помітно випереджав темп приросту нових користувачів?
3. Чи можна, дивлячись на графіки з відносними показниками, сказати, що кількість нових кур'єрів більш стабільний, ніж показник числа нових користувачів?

## Задача 3

Тепер пропонуємо вам подивитися на нашу аудиторію трохи під іншим кутом - давайте порахуємо не просто всіх користувачів, а саме ту частину, яка оформлює та оплачує замовлення у нашому сервісі. Заодно з'ясуємо, яку частку користувачі, що платять, становлять від їх загального числа.

::::: {.callout-note icon=false}
## Завдання
:::: {#exr-sql-analytic-03}
<br>
Для кожного дня, представленого в таблицях `user_actions` та `courier_actions`, розрахуйте такі показники:

1. Число користувачів, які оплачують замовлення.
2. Число активних кур'єрів.
3. Частку користувачів, що платять, в загальній кількості користувачів на поточний день.
4. Частку активних кур'єрів у кількості кур'єрів на поточний день.

Колонки з показниками назвіть відповідно `paying_users`, `active_couriers`, `paying_users_share`, `active_couriers_share`. Колонку з датами назвіть `date`. Простежте, щоб абсолютні показники були виражені **цілими числами**. Усі показники часток необхідно виразити у **відсотках**. Під час їх розрахунку округляйте значення **до двох знаків після коми**.

Результат має бути відсортований за зростанням дати.

Поля в результуючій таблиці: `date`, `paying_users`, `active_couriers`, `paying_users_share`, `active_couriers_share`

**Пояснення:**

Користувачів, які оплачують замовлення будемо вважати тих, які в даний день оформили хоча б одне замовлення, яке надалі не було скасовано.

Кур'єрів вважатимемо активними, якщо в даний день вони прийняли хоча б одне замовлення, яке було доставлено (можливо вже наступного дня), або доставили будь-яке замовлення.

Загальна кількість користувачів/кур'єрів на поточний день – це, як і раніше, результат складання числа нових користувачів/кур'єрів у поточний день зі значеннями аналогічного показника всіх попередніх днів. Ми рахували цей показник на попередніх кроках.

Не забувайте при діленні заздалегідь наводити значення до потрібного типу даних.

::: {.callout-note collapse="true"}
## Підказка на випадок, якщо зовсім не виходить

Для вирішення завдання необхідно спочатку для кожного дня порахувати кількість користувачів, які оплачують замовлення, і активних кур'єрів, а потім об'єднати отримані таблиці з даними про загальну кількість користувачів і кур'єрів на поточну дату і далі розрахувати всі необхідні відносні показники. Загальну кількість користувачів та кур'єрів на поточну дату ми вже рахували раніше.
:::


::::
:::::

In [8]:
#| code-fold: true
#| code-summary: "Рішення"

%%sql
SELECT date,
       paying_users,
       active_couriers,
       round(100 * paying_users::decimal / total_users, 2) as paying_users_share,
       round(100 * active_couriers::decimal / total_couriers, 2) as active_couriers_share
FROM   (SELECT start_date as date,
               new_users,
               new_couriers,
               (sum(new_users) OVER (ORDER BY start_date))::int as total_users,
               (sum(new_couriers) OVER (ORDER BY start_date))::int as total_couriers
        FROM   (SELECT start_date,
                       count(courier_id) as new_couriers
                FROM   (SELECT courier_id,
                               min(time::date) as start_date
                        FROM   courier_actions
                        GROUP BY courier_id) t1
                GROUP BY start_date) t2
            LEFT JOIN (SELECT start_date,
                              count(user_id) as new_users
                       FROM   (SELECT user_id,
                                      min(time::date) as start_date
                               FROM   user_actions
                               GROUP BY user_id) t3
                       GROUP BY start_date) t4 using (start_date)) t5
    LEFT JOIN (SELECT time::date as date,
                      count(distinct courier_id) as active_couriers
               FROM   courier_actions
               WHERE  order_id not in (SELECT order_id
                                       FROM   user_actions
                                       WHERE  action = 'cancel_order')
               GROUP BY date) t6 using (date)
    LEFT JOIN (SELECT time::date as date,
                      count(distinct user_id) as paying_users
               FROM   user_actions
               WHERE  order_id not in (SELECT order_id
                                       FROM   user_actions
                                       WHERE  action = 'cancel_order')
               GROUP BY date) t7 using (date)

,date,paying_users,active_couriers,paying_users_share,active_couriers_share
0,2022-08-24,127,93,94.78,97.89
1,2022-08-25,820,333,85.15,98.81
2,2022-08-26,1137,526,57.42,94.60
3,2022-08-27,1548,721,48.30,97.17
4,2022-08-28,2083,927,44.39,97.07
5,2022-08-29,2202,1022,36.69,96.05
6,2022-08-30,2253,1133,31.27,95.13
7,2022-08-31,2430,1286,28.17,92.78
8,2022-09-01,2710,1474,26.73,92.65
9,2022-09-02,3429,1733,28.52,95.22


Запишемо результат запиту у змінну `results`.

In [9]:
#| code-fold: true

%%sql
results << SELECT date,
       paying_users,
       active_couriers,
       round(100 * paying_users::decimal / total_users, 2) as paying_users_share,
       round(100 * active_couriers::decimal / total_couriers, 2) as active_couriers_share
FROM   (SELECT start_date as date,
               new_users,
               new_couriers,
               (sum(new_users) OVER (ORDER BY start_date))::int as total_users,
               (sum(new_couriers) OVER (ORDER BY start_date))::int as total_couriers
        FROM   (SELECT start_date,
                       count(courier_id) as new_couriers
                FROM   (SELECT courier_id,
                               min(time::date) as start_date
                        FROM   courier_actions
                        GROUP BY courier_id) t1
                GROUP BY start_date) t2
            LEFT JOIN (SELECT start_date,
                              count(user_id) as new_users
                       FROM   (SELECT user_id,
                                      min(time::date) as start_date
                               FROM   user_actions
                               GROUP BY user_id) t3
                       GROUP BY start_date) t4 using (start_date)) t5
    LEFT JOIN (SELECT time::date as date,
                      count(distinct courier_id) as active_couriers
               FROM   courier_actions
               WHERE  order_id not in (SELECT order_id
                                       FROM   user_actions
                                       WHERE  action = 'cancel_order')
               GROUP BY date) t6 using (date)
    LEFT JOIN (SELECT time::date as date,
                      count(distinct user_id) as paying_users
               FROM   user_actions
               WHERE  order_id not in (SELECT order_id
                                       FROM   user_actions
                                       WHERE  action = 'cancel_order')
               GROUP BY date) t7 using (date)

Побудуємо візуалізацію за отриманими даними:

In [10]:
# | label: fig-sql-analytic-03
# | fig-cap: "Графік за результатами SQL-запиту"
# | fig-subcap:
# |   - "Динаміка користувачів, які платять та активних кур'єрів:"
# |   - "Динаміка часток користувачів, які платять та активних кур'єрів:"
# | column: screen-inset-shaded
# | layout-nrow: 1

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=results.date,
        y=results.paying_users,
        mode="lines+markers",
        name="paying_users",
    )
)
fig.add_trace(
    go.Scatter(
        x=results.date,
        y=results.active_couriers,
        mode="lines+markers",
        name="active_couriers",
    )
)
fig.show()

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=results.date,
        y=results.paying_users_share,
        mode="lines+markers",
        name="paying_users_share",
    )
)
fig.add_trace(
    go.Scatter(
        x=results.date,
        y=results.active_couriers_share,
        mode="lines+markers",
        name="active_couriers_share",
    )
)
fig.show()

Проаналізуйте побудовані графіки та спробуйте відповісти на такі питання:

1. Чи можна сказати, що разом із загальною кількістю користувачів і кур'єрів зростає кількість користувачів, що платять, і активних кур'єрів?
2. Як водночас поводяться показники часток користувачів, що платять, і активних кур'єрів? Чи можна вважати їхню поточну динаміку в цілому нормальною та закономірною?

## Задача 4

Давайте докладніше зупинимося на користувачах, які оплачують замовлення, копнем трохи глибше і з'ясуємо, як багато користувачів, які оплачують замовленняь, здійснюють більше одного замовлення в день. Зрештою нам важливо розуміти, як здебільшого поводяться наші користувачі — вони заходять у додаток, щоб зробити всього одне замовлення, або наш сервіс настільки хороший, що вони готові користуватися ним кілька разів на день.

::::: {.callout-note icon=false}
## Завдання
:::: {#exr-sql-analytic-04}
<br>
Для кожного дня, поданого в таблиці `user_actions`, розрахуйте такі показники:

1. Частку користувачів, які зробили в цей день всього одне замовлення, від загальної кількості користувачів, які оплачують замовлення.
2. Частку користувачів, які зробили цього дня кілька замовлень, від загальної кількості користувачів, які оплачують замовлення.

Назвіть колонки з показниками відповідно `single_order_users_share`, `several_orders_users_share`. Колонку з датами назвіть `date`. Усі показники із частками необхідно виразити **у відсотках**. При розрахунку часток округляйте значення **до двох знаків після коми**.

Результат має бути відсортований за зростанням дати.

Поля в результуючій таблиці: `date`, `single_order_users_share`, `several_orders_users_share`

**Пояснення:**

Користувачами, що оплачують замовлення, як і раніше, вважаємо тих користувачів, які в даний день оформили (і не скасували) хоча б одне замовлення.

Не забувайте при діленні заздалегідь наводити значення до потрібного типу даних.

::: {.callout-note collapse="true"}
## Підказка на випадок, якщо зовсім не виходить

Для вирішення завдання спочатку необхідно для кожного дня окремо розрахувати загальну кількість користувачів, що оплачують замовлення, кількість користувачів з одним замовленням і кількість користувачів з кількома замовленнями. Потім необхідно поєднати таблиці з цими даними за датою та порахувати всі необхідні показники.
:::

::::
:::::

In [11]:
#| code-fold: true
#| code-summary: "Рішення"
#| column: page-inset

%%sql
SELECT day::date AS date,
       ROUND(100 * COUNT(DISTINCT user_id) FILTER(WHERE count_orders = 1) / COUNT(DISTINCT user_id)::numeric, 2) AS single_order_users_share,
       ROUND(100 * COUNT(DISTINCT user_id) FILTER(WHERE count_orders != 1) / COUNT(DISTINCT user_id)::numeric, 2) AS several_orders_users_share
FROM(

SELECT user_id,
       COUNT(DISTINCT order_id) AS count_orders,
       DATE_TRUNC('day', time) AS day
FROM user_actions
WHERE order_id NOT IN (SELECT order_id FROM user_actions WHERE action = 'cancel_order')
GROUP BY user_id, day
) AS t1
GROUP BY day
ORDER BY day

,date,single_order_users_share,several_orders_users_share
0,2022-08-24,92.91,7.09
1,2022-08-25,76.46,23.54
2,2022-08-26,78.45,21.55
3,2022-08-27,70.93,29.07
4,2022-08-28,67.83,32.17
5,2022-08-29,65.58,34.42
6,2022-08-30,64.67,35.33
7,2022-08-31,70.41,29.59
8,2022-09-01,71.70,28.30
9,2022-09-02,67.02,32.98


Запишемо результат запиту у змінну `results`.

In [12]:
#| code-fold: true

%%sql
results << SELECT day::date AS date,
       ROUND(100 * COUNT(DISTINCT user_id) FILTER(WHERE count_orders = 1) / COUNT(DISTINCT user_id)::numeric, 2) AS single_order_users_share,
       ROUND(100 * COUNT(DISTINCT user_id) FILTER(WHERE count_orders != 1) / COUNT(DISTINCT user_id)::numeric, 2) AS several_orders_users_share
FROM(

SELECT user_id,
       COUNT(DISTINCT order_id) AS count_orders,
       DATE_TRUNC('day', time) AS day
FROM user_actions
WHERE order_id NOT IN (SELECT order_id FROM user_actions WHERE action = 'cancel_order')
GROUP BY user_id, day
) AS t1
GROUP BY day
ORDER BY day

Побудуємо візуалізацію за отриманими даними:

In [13]:
# | label: fig-sql-analytic-04
# | fig-cap: "Частки користувачів з одним та кількома замовленнями"

fig = go.Figure(
    data=[
        go.Bar(
            name="single_order_users_share",
            x=results.date,
            y=results.single_order_users_share,
        ),
        go.Bar(
            name="several_orders_users_share",
            x=results.date,
            y=results.several_orders_users_share,
        ),
    ]
)

fig.update_layout(barmode="stack")
fig.show()

Проаналізуйте побудовані графіки та спробуйте відповісти на такі питання:

1. На якому рівні у середньому тримається частка користувачів із кількома замовленнями?
2. Чи можна вважати значення показника в перший день аномально низьким, якщо взяти до уваги загальну кількість користувачів цього дня?